# AUTOMATIC TICKET ASSIGNMENT

Importing Libraries

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,scale
from data_eng_utils import preText, lemmatizer

In [72]:
import os
import torch
from transformers import BertTokenizer

In [33]:
data='DATA'
raw_data_folder='RAW_DATA'
processed_data_folder='PROCESSED_DATA'

In [34]:
if data not in os.listdir(os.getcwd()):
    os.makedirs(os.path.join(data))
DATA_PATH=os.path.join(data)
if raw_data_folder not in os.listdir(os.path.join(DATA_PATH)):
    os.makedirs(os.path.join(DATA_PATH,raw_data_folder))
RAW_DATA_PATH=os.path.join(DATA_PATH,raw_data_folder)
if processed_data_folder not in os.listdir(os.path.join(DATA_PATH)):
    os.makedirs(os.path.join(DATA_PATH,processed_data_folder))
PROCESSED_DATA_PATH=os.path.join(DATA_PATH,processed_data_folder)

Read the csv file and see some records

In [35]:
new_df=pd.read_csv(os.path.join(RAW_DATA_PATH,'Input Data Translated.csv'), encoding = 'latin-1')
new_df.head()

,Short description,Description,Caller,Assignment group,Lang_Short_Description,Lang_Description
0,login issue,-verified user details.(employee# & manager na...,spxjnwir pjlcoqds,GRP_0,en,en
1,outlook,\n\nreceived from: hmjdrvpb.komuaywn@gmail.com...,hmjdrvpb komuaywn,GRP_0,en,en
2,cant log in to vpn,\n\nreceived from: eylqgodm.ybqkwiam@gmail.com...,eylqgodm ybqkwiam,GRP_0,en,en
3,unable to access hr_tool page,unable to access hr_tool page,xbkucsvz gcpydteq,GRP_0,en,en
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0,en,en


In [36]:
new_df.shape

(8500, 6)

In [37]:
# NULL replacement
new_df.fillna(str(), inplace=True)
new_df[pd.isnull(new_df).any(axis=1)]

,Short description,Description,Caller,Assignment group,Lang_Short_Description,Lang_Description


Now there is no null value in any column

Checking for Duplicates across Short Description and Description

In [38]:
df_common=new_df[new_df[["Short description","Description"]].apply(lambda x : x[0]==x[1],axis=1)].reset_index(drop=True).copy()

In [39]:
df_common.head()

,Short description,Description,Caller,Assignment group,Lang_Short_Description,Lang_Description
0,unable to access hr_tool page,unable to access hr_tool page,xbkucsvz gcpydteq,GRP_0,en,en
1,skype error,skype error,owlgqjme qhcozdfx,GRP_0,en,en
2,unable to log in to engineering tool and skype,unable to log in to engineering tool and skype,eflahbxn ltdgrvkz,GRP_0,en,en
3,ticket_no1550391- employment status - new non-...,ticket_no1550391- employment status - new non-...,eqzibjhw ymebpoih,GRP_0,en,en
4,unable to disable add ins on outlook,unable to disable add ins on outlook,mdbegvct dbvichlg,GRP_0,en,en


In [40]:
df_common.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2889 entries, 0 to 2888
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Short description       2889 non-null   object
 1   Description             2889 non-null   object
 2   Caller                  2889 non-null   object
 3   Assignment group        2889 non-null   object
 4   Lang_Short_Description  2889 non-null   object
 5   Lang_Description        2889 non-null   object
dtypes: object(6)
memory usage: 135.5+ KB


Comments: this shows that out of the 8500 records, 2889 have the same information in both Short description and Description - This would point out that it is better to concatenate the two columns for the modeling stage

Language as a column is not required for the modeling but was just for understanding; dropping the Language Column

# Next Steps

# Data Cleaning 

In this step there is a need to 
1. Convert all text to lower case
2. Remove numbers
3. Remove puntuations
4. Remove blank spaces
5. Remove stop words (along with other words identified earlier that would not contribute)
6. Remove email id's

In [41]:
import re # for applying Regex pattern to subject strings

In [42]:
# Fixing the different patterns
email_pat = r"([\w.+-]+@[a-z\d-]+\.[a-z\d.-]+)"
punct_pat = r"[,|.|_|@|\|?|\\|$&*|%|\r|\n|.:|\s+|/|//|\\|/|\||-|<|>|;|(|)|=|+|#|-|\"|[-\]]|{|}]"
num_pat = r"(?<!)(\d+(?:\.\d+)?)"


In [43]:
# Checking to see how the cleaning function has worked for a record
print('\033[1mOriginal text:\033[0m')
print(new_df['Description'][0])
print('_'*100)
print('\033[1mCleaned text:\033[0m')
print(preText(new_df['Description'][0]))

Original text:
-verified user details.(employee# & manager name)
-checked the user name in ad and reset the password.
-advised the user to login and check.
-caller confirmed that he was able to login.
-issue resolved.
____________________________________________________________________________________________________
Cleaned text:
verified user details employee manager name checked the user name in ad and reset the password advised the user to login and check caller confirmed that he was able to login issue resolved


Now given that The text seems to have been pre-processed correctly, applying it on the total database

In [44]:
new_df['Description'] = new_df['Description'].apply(preText)
new_df['Short description'] = new_df['Short description'].apply(preText)

# Verify the data
new_df.head()

,Short description,Description,Caller,Assignment group,Lang_Short_Description,Lang_Description
0,login issue,verified user details employee manager name ch...,spxjnwir pjlcoqds,GRP_0,en,en
1,outlook,received from hello team my meetings skype mee...,hmjdrvpb komuaywn,GRP_0,en,en
2,cant log in to vpn,received from hi i cannot log on to vpn best,eylqgodm ybqkwiam,GRP_0,en,en
3,unable to access hr tool page,unable to access hr tool page,xbkucsvz gcpydteq,GRP_0,en,en
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0,en,en


# Text Pre Processing

As first steps, we would be concatenating Short Description and Description columns (given that more than a fourth of records have exactly the same data in both of them)

In [45]:
new_df.insert(loc=4, 
              column='Total', 
              allow_duplicates=True, 
              value=list(new_df['Short description'].str.strip() + ' ' + new_df['Description'].str.strip()))

In [46]:
new_df.head()

,Short description,Description,Caller,Assignment group,Total,Lang_Short_Description,Lang_Description
0,login issue,verified user details employee manager name ch...,spxjnwir pjlcoqds,GRP_0,login issue verified user details employee man...,en,en
1,outlook,received from hello team my meetings skype mee...,hmjdrvpb komuaywn,GRP_0,outlook received from hello team my meetings s...,en,en
2,cant log in to vpn,received from hi i cannot log on to vpn best,eylqgodm ybqkwiam,GRP_0,cant log in to vpn received from hi i cannot l...,en,en
3,unable to access hr tool page,unable to access hr tool page,xbkucsvz gcpydteq,GRP_0,unable to access hr tool page unable to access...,en,en
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0,skype error skype error,en,en


Comments: As seen in case 4, there is duplication and this needs to be removed

In [47]:
new_df["Total"] = new_df["Total"].apply(lambda x: ' '.join(pd.unique(x.split()))) # removing duplicates

In [48]:
new_df.head()

,Short description,Description,Caller,Assignment group,Total,Lang_Short_Description,Lang_Description
0,login issue,verified user details employee manager name ch...,spxjnwir pjlcoqds,GRP_0,login issue verified user details employee man...,en,en
1,outlook,received from hello team my meetings skype mee...,hmjdrvpb komuaywn,GRP_0,outlook received from hello team my meetings s...,en,en
2,cant log in to vpn,received from hi i cannot log on to vpn best,eylqgodm ybqkwiam,GRP_0,cant log in to vpn received from hi i cannot o...,en,en
3,unable to access hr tool page,unable to access hr tool page,xbkucsvz gcpydteq,GRP_0,unable to access hr tool page,en,en
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0,skype error,en,en


# Lemmatization

We are using spaCY for this given that it also takes of POS and works well on cleaned data

In [50]:
import spacy

In [51]:
import sys
!{sys.executable} -m spacy download en

     -------------------------------------- 12.8/12.8 MB 442.6 kB/s eta 0:00:00
[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [53]:
# Initialize spacy 'en' medium model, keeping only tagger component needed for lemmatization
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [54]:
# Checking to see how the lemmatizer function has worked for a record
print('\033[1mOriginal text:\033[0m')
print(new_df['Total'][0])
print('_'*100)
print('\033[1mLemmatized text:\033[0m')
print(lemmatizer(new_df['Total'][0]))

Original text:
login issue verified user details employee manager name checked the in ad and reset password advised to check caller confirmed that he was able resolved
____________________________________________________________________________________________________
Lemmatized text:
login issue verify user detail employee manager name check the in ad and reset password advise to check caller confirm that he be able resolve


In [55]:
# Applying on the database
new_df['Total'] = new_df['Total'].apply(lemmatizer)

# Verify the data
new_df.head()

,Short description,Description,Caller,Assignment group,Total,Lang_Short_Description,Lang_Description
0,login issue,verified user details employee manager name ch...,spxjnwir pjlcoqds,GRP_0,login issue verify user detail employee manage...,en,en
1,outlook,received from hello team my meetings skype mee...,hmjdrvpb komuaywn,GRP_0,outlook receive from hello team my meeting sky...,en,en
2,cant log in to vpn,received from hi i cannot log on to vpn best,eylqgodm ybqkwiam,GRP_0,can not log in to vpn receive from hi I can no...,en,en
3,unable to access hr tool page,unable to access hr tool page,xbkucsvz gcpydteq,GRP_0,unable to access hr tool page,en,en
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0,skype error,en,en


# Preparing list of Stop words

In [56]:
import nltk as nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

     ---------------------------------------- 1.5/1.5 MB 425.7 kB/s eta 0:00:00


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Swami\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [57]:
allstp=np.array(stopwords.words('english'))
allstp.size

179

In [58]:
#Creating an additional of stopwords that  we see as irrelevant to the modelling inputs
new_words=np.array(['yes','hi', 'receive','hello','sir','madam', 'best','morning','evening','afternoon' 'regards','thanks','from','greeting', 'forward','reply','will','please','see','help','able'])
new_words.size

20

In [59]:
stopwords=np.concatenate([allstp,new_words]) #Concatenating nltk list and our list of stopwords
stopwords.size

199

In [60]:
index_not = np.where(stopwords == 'not') # find index of 'not' in the stopwords
index_not

(array([118], dtype=int64),)

In [61]:
final_list=np.delete(stopwords, index_not)
final_list.size

198

In [62]:
#the cleaning process below removes the stopwords defined above as a string irrespective of whether it is part of another word. Example: it will remove "i" in input as "i" is a stopword. Hence, to prevent the same, we will append a space before and after every term to defined as a word
final_list1=[]
for i in final_list:
#     print("initial")
#     print(i)
    txt=i
    txt=" "+txt+" "
    i=txt
    final_list1.append(i)
#     print("final:")
#     print(i)
# final_list1

## Cleaning usernames and stop words from the descriptions

In [63]:
uniq=new_df['Caller'].unique()
print(uniq)
uniq.size

['spxjnwir pjlcoqds' 'hmjdrvpb komuaywn' 'eylqgodm ybqkwiam' ...
 'bjitvswa yrmugfnq' 'oybwdsgx oxyhwrfz' 'kqvbrspl jyzoklfx']


2950

In [64]:
## We can use the same command for removing whatever stop words we want. Will append those terms to the uniq array using:
## np.append(uniq,stopword)

In [65]:
uniq1=np.concatenate([uniq,final_list1]) #preparing final list of terms that need to be deleted. This includes usernames and stopwords
uniq1.size

3148

In [66]:
new_df['Clean Description']=new_df['Total'].copy()

In [68]:
# print commands have been commented out due to issues in the IOrate. You can uncomment out the same in case you need the reference
s=" "
for key, value in new_df['Total'].items():
    r=value
    
    if(pd.isnull(value)):
        s=''
    else:
#         print(key)
        for u in range(uniq1.size):
            if(r.find(uniq1[u])!=-1):
            
        #print(uniq[u])
#                 print('un found: ',uniq1[u])
                s = r.replace(uniq1[u],' ')
                r=s
#                 print('Key: ',key)
#                 print('Original string: ',r)
#                 print('Revised string: ',s)
            else:
                s=r
#     print(key)
#     print(r)
#     print(s)
    new_df.at[key,'Clean Description']= s
#     print(key)

In [69]:
new_df.head()

,Short description,Description,Caller,Assignment group,Total,Lang_Short_Description,Lang_Description,Clean Description
0,login issue,verified user details employee manager name ch...,spxjnwir pjlcoqds,GRP_0,login issue verify user detail employee manage...,en,en,login issue verify user detail employee manage...
1,outlook,received from hello team my meetings skype mee...,hmjdrvpb komuaywn,GRP_0,outlook receive from hello team my meeting sky...,en,en,outlook team meeting skype etc not appear cale...
2,cant log in to vpn,received from hi i cannot log on to vpn best,eylqgodm ybqkwiam,GRP_0,can not log in to vpn receive from hi I can no...,en,en,can not log vpn I not good
3,unable to access hr tool page,unable to access hr tool page,xbkucsvz gcpydteq,GRP_0,unable to access hr tool page,en,en,unable access hr tool page
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0,skype error,en,en,skype error


Comments: Now the Column "Clean Description" has completely clean data where all the stop words, names of people removed and we can explore this data

In [71]:
features=new_df['Clean Description']

In [75]:
features.to_csv(os.path.join(RAW_DATA_PATH,'Features.csv'))